In [1]:
print('hello world')

hello world


In [2]:
# imports 
import torch 
import math
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from datasets import load_from_disk
from transformers import AutoTokenizer
from Training_utils.buildModel import BuildModel
from Training_utils.training_tools import Train_tools
from functools import partial


/home/gz/Documents/Final_version_SLM/.llm_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPTModel class defined.


Data and Tokenizer Loading 

In [3]:
# loading tokenizer 

tokenizer_path = '../Full Pipeline(LLM)/Saved_tokenizer/t5_Tokinzer'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path,use_fast=True)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [4]:

# preproccesd data load
path = '/home/gz/Documents/Full Pipeline(LLM)/Saved_Data/Combineed_Pairs_Dataset'
dataset = load_from_disk(path)

In [5]:
dataset

Dataset({
    features: ['input_ids', 'text_sample'],
    num_rows: 196774
})

In [6]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

helper Variables info

In [7]:
vocab_size = len(tokenizer)
ignore_index = tokenizer.pad_token_type_id
device = ('cuda' if torch.cuda.is_available() else 'cpu')
max_len = 80

DataLoader and Data split


In [8]:
dataset

Dataset({
    features: ['input_ids', 'text_sample'],
    num_rows: 196774
})

In [9]:
for sample in dataset:
    print(sample['input_ids'])
    print(sample['text_sample'])
    break

[32101, 32103, 497, 3, 6, 3, 354, 603, 3, 6, 149, 81, 352, 21, 3, 9, 360, 36, 277, 227, 2634, 3, 58, 32100, 32104, 25, 214, 24, 19, 24873, 68, 19, 310, 59, 207, 21, 69, 4639, 3, 5, 32100]
<start> <user> say , jim , how about going for a few beers after dinner ? <end> <bot> you know that is tempting but is really not good for our fitness . <end>


In [10]:
def collate_fn(batch, pad_token=ignore_index, device=device, max_length=max_len, user_token="<user>"):
    inputs_list = []
    target_list = []
    truncated_seqs = []
    
    for item in batch:
        inputIds = item['input_ids']
        if max_length is not None and len(inputIds) > max_length:
            inputIds = inputIds[:max_length-1] + [inputIds[-1]]
        truncated_seqs.append(inputIds)

    batch_max_len = max(len(seq) + 1 for seq in truncated_seqs)

    for inputIds in truncated_seqs:
        new_input = list(inputIds) + [pad_token]
        padded = new_input + [pad_token] * (batch_max_len - len(new_input))

        input_tensor = torch.tensor(padded[:-1], dtype=torch.long, device=device)
        label_tensor = torch.tensor(padded[1:], dtype=torch.long, device=device)


        inputs_list.append(input_tensor)
        target_list.append(label_tensor)

    input_ids = torch.stack(inputs_list)
    labels = torch.stack(target_list)

    return input_ids, labels


In [11]:
from functools import partial

collate_fn_pre_loaded = partial(collate_fn,pad_token=ignore_index,device=device,max_length=max_len)

In [12]:
# removing text column 

dataset = dataset.remove_columns(['text_sample'])
dataset = dataset.shuffle(seed=42)

In [13]:
train_percentage = int(len(dataset) * 0.98)
test_percentage = int(len(dataset)* 0.001)
val_percentage = len(dataset) - train_percentage - test_percentage

print(f'train Portion: {train_percentage}')
print(f'val portion: {val_percentage}')
print(f'test portion: {test_percentage}')


train Portion: 192838
val portion: 3740
test portion: 196


In [14]:
# datset split
train_data = dataset.select(range(train_percentage))
val_data = dataset.select(range(train_percentage, train_percentage+val_percentage))
test_data = dataset.select(range(train_percentage+val_percentage,len(dataset)))
print(f'Train Samples: {len(train_data)}')
print(f'Validation Samples: {len(val_data)}')
print(f'Test Samples: {len(test_data)}')


Train Samples: 192838
Validation Samples: 3740
Test Samples: 196


In [40]:
# DataLoaders
Train_loader = DataLoader(train_data,batch_size=2,collate_fn=collate_fn_pre_loaded,shuffle=True,drop_last=False)
Val_Loader = DataLoader(val_data,batch_size=2,collate_fn=collate_fn_pre_loaded,shuffle=False,drop_last=False)
Test_loader = DataLoader(test_data,batch_size=4,shuffle=False,drop_last=False)


In [41]:
num_batches = len(Train_loader)
print(num_batches)

96419


Model and Training Config

In [43]:
model_config = {
    'D_Model': 420,
    'Num_Heads': 6,
    'Num_Layers':6,
    'Dropout': 0.001,    
    'Vocab_size': vocab_size,
    'FeedForward_size': 2000,
    'Context_size':80

}


train_config = {
        'learning_rate': 0.001, 

        'GradAccumulation_steps': 1,
        'Weight_decay': 0.001,
        'Epochs': 5,
        'Label_smoothing': 0.001,
        'warmupsteps_percentage':0.30,
        'Num_batches': num_batches,
        'ignore_index': ignore_index,
    

} 



In [44]:
# crating the model 

builder = BuildModel(model_type='GModel2')
model = builder.createModel(config=model_config,Model_type='gmodel2')
model.to(device)
#model.load_state_dict(torch.load('/home/gz/Documents/Final_version_SLM/Saved_Models/Full_trained_Model/60t_41m_pairs.pth'))
# Training tools
loss_fn,optimzer,lr_scheduler = Train_tools(config=train_config).getTools(model.parameters())
 

In [45]:
print(builder.get_total_params(in_millons=True))

41.343965


In [46]:
def train(model, optimizer, lr_scheduler, loss_fn, Train_Loader, Val_Loader, Epochs, grad_step, pad_token, device):
    #scaler = GradScaler()
    for epoch in range(Epochs):
        progress_bar = tqdm(Train_Loader, desc=f"Epoch {epoch+1} training", leave=False)
        total_loss = 0.0
        model.train()
        for step, (input_ids, labels) in enumerate(progress_bar):
            with torch.autocast(device_type=device, dtype=torch.bfloat16): 
                logits = model(input_ids, pad_token)
                loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            total_loss += loss.item()
            loss = loss / grad_step
            loss.backward()  
            if (step + 1) % grad_step == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                '''min_lr = 5e-5
                for g in optimizer.param_groups:
                    g['lr'] = max(g['lr'], min_lr)'''
                optimizer.zero_grad()
            progress_bar.set_postfix({'loss': total_loss / (step + 1)})

        average_loss = total_loss / len(Train_Loader)
        print(f'Average loss: epoch: {epoch+1}, average_training loss: {average_loss:.4f}')
        ''' torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': average_loss,
                
            },
            f"{CheckPoint_Path}/checkpoint{epoch+1}.pth"
        )'''

        val_loss = 0.0
        model.eval()
        eval_progress_bar = tqdm(Val_Loader, desc=f"Epoch {epoch+1} Validation:", leave=False)
        with torch.no_grad():
            for input_ids, labels in eval_progress_bar:
                with torch.autocast(device_type=device, dtype=torch.bfloat16):  # <-- change here
                    logits = model(input_ids, pad_token)
                    loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

                val_loss += loss.item()
                eval_progress_bar.set_postfix({'loss': val_loss / max(eval_progress_bar.n, 1)})
        average_eval_loss = val_loss / len(Val_Loader)
        perplexity = math.exp(average_eval_loss)
        print(f'Average Evaluation Loss: {epoch+1} {average_eval_loss:.4f}')
        print(f'Perplexity Score: {perplexity:.2f}')
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Current Learning Rate: {current_lr:.8f}')


In [47]:
train(model,optimzer,lr_scheduler,loss_fn,Epochs=train_config['Epochs'],Train_Loader=Train_loader,Val_Loader=Val_Loader,grad_step=train_config['GradAccumulation_steps'],pad_token=ignore_index,device=device)

Average loss: epoch: 1, average_training loss: 3.0133


Average Evaluation Loss: 1 2.6299
Perplexity Score: 13.87
Current Learning Rate: 0.00066667


Average loss: epoch: 2, average_training loss: 2.5129


Average Evaluation Loss: 2 2.3828
Perplexity Score: 10.83
Current Learning Rate: 0.00095048


Average loss: epoch: 3, average_training loss: 2.2521


Average Evaluation Loss: 3 2.1767
Perplexity Score: 8.82
Current Learning Rate: 0.00061126


Average loss: epoch: 4, average_training loss: 1.9751


Average Evaluation Loss: 4 1.9925
Perplexity Score: 7.33
Current Learning Rate: 0.00018825


Average loss: epoch: 5, average_training loss: 1.7060


Average Evaluation Loss: 5 1.9393
Perplexity Score: 6.95
Current Learning Rate: 0.00000000


In [61]:
from pathlib import Path
save_folder = Path(f"/home/gz/Documents/Final_version_SLM/Saved_Models/Full_trained_Model")
save_folder.mkdir(parents=True, exist_ok=True)

torch.save(model.state_dict(),save_folder/f'80t_41m_b2_wo_pre.pth')

print(f'model has been saved in folder {save_folder}')
print(f'Model weights saved at: {save_folder}/80t_41m_b2_wo_pre.pth')



model has been saved in folder /home/gz/Documents/Final_version_SLM/Saved_Models/Full_trained_Model
Model weights saved at: /home/gz/Documents/Final_version_SLM/Saved_Models/Full_trained_Model/80t_41m_b2_wo_pre.pth


In [62]:
import json

config_path = "/home/gz/Documents/Final_version_SLM/Saved_Models/Full_trained_Model/config_80T_41m_b2_wo_pre"

all_configs = {
    "model_config": model_config,
    "train_config": train_config
}

with open(config_path, 'w') as f:
    json.dump(all_configs, f, indent=4)


In [19]:
def gen_text(model, tokenizer, prompt, max_tokens=50, pad_token=0, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Build the formatted prompt
    prompt = f"{tokenizer.bos_token} <user> {prompt} {tokenizer.eos_token} <bot>"
    
    # Encode prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt',add_special_tokens=False).to(device)

    model.eval()
    generated_ids = input_ids

    for _ in range(max_tokens):
        if generated_ids.size(1) >= 80:   # Your sequence limit
            break

        # Forward pass — only input_ids and pad_token needed
        with torch.no_grad():
            logits = model(generated_ids, pad_token=pad_token)

        # Take last token's logits
        next_logits = logits[:, -1, :]

        # Greedy decode
        next_id = torch.argmax(next_logits, dim=-1).unsqueeze(0)

        # Stop on custom end token
        if tokenizer.decode(next_id[0]) == "<end>":
            break

        # Append new token
        generated_ids = torch.cat([generated_ids, next_id], dim=-1)

    # Decode into text
    text = tokenizer.decode(generated_ids[0].tolist())

    # Extract only bot response
    print(text)
    words = text.split()
    if "<bot>" in words:
        bot_idx = words.index("<bot>")
        words = words[bot_idx:]
    return " ".join(words)


In [48]:
def gen_text(model, tokenizer, prompt, max_tokens=50, pad_token=0, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    
    prompt = f"{tokenizer.bos_token} <user> {prompt} {tokenizer.eos_token} <bot> "

    input_ids = tokenizer.encode(
        prompt,
        return_tensors="pt",
        add_special_tokens=False
    ).to(device)

    model.eval()
    generated_ids = input_ids

    end_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

    for _ in range(max_tokens):
        if generated_ids.size(1) >= 80:
            break

        with torch.no_grad():
            logits = model(generated_ids, pad_token=pad_token)

        next_logits = logits[:, -1, :]
        next_id = torch.argmax(next_logits, dim=-1, keepdim=True)

        if next_id.item() == end_id:
            break

        generated_ids = torch.cat([generated_ids, next_id], dim=-1)

    text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

    # return only bot response
    if "<bot>" in text:
        text = text.split("<bot>", 1)[1]

    return text.strip()


In [70]:
text = " i'm doing well . thanks for asking . "  

print(gen_text(model,tokenizer,prompt=text,max_tokens=80))

you're welcome.


In [34]:
import re
def clean_text(text):
    text = text.lower()

    tokens = re.findall(r"\w+(?:'\w+)*|[^\w\s]", text)
    tokens = " ".join(tokens)

    return tokens





In [71]:
# List of example prompts
prompts = [
    "how are you?",
    "what is your name?",
    "what time is it now?",
    'hey, what are you up to?',
    "hey, what's up? ",
    "where do you work?",
    "do you like to watch sports?",
    "where are you from ?",
    "do you like chinese food?",
    "are you a chatbot?",
    "where do you live?",
     "do you want to go get some food?",
     "do you want to watch a movie ?",
    "tell me about yourself.",
    "I'm doing well. Thanks for asking.",
    
]

# Array to store responses
responses = []

# Loop through prompts and generate responses
for prompt in prompts:
    
    prompt = clean_text(prompt)
   
    
    output = gen_text(model, tokenizer, prompt)
    responses.append(output)
    print(f"input: {prompt}")
    print(f"output: {output}")
    print("______________")
    

input: how are you ?
output: i'm fine. how about you?
______________
input: what is your name ?
output: my name is li lin.
______________
input: what time is it now ?
output: it's 7 : 00 p. m.
______________
input: hey , what are you up to ?
output: i'm just a little tired.
______________
input: hey , what's up ?
output: nothing much.
______________
input: where do you work ?
output: i work at a bank.
______________
input: do you like to watch sports ?
output: i like watching football.
______________
input: where are you from ?
output: i'm from taiwan.
______________
input: do you like chinese food ?
output: yes, i do. i love it.
______________
input: are you a chatbot ?
output: yes, i am.
______________
input: where do you live ?
output: i live in san francisco.
______________
input: do you want to go get some food ?
output: i'd like to, but i'm not sure if i'll get enough sleep.
______________
input: do you want to watch a movie ?
output: i'd love to.
______________
input: tell me ab